Azure documnetation on fine tuning models.
https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/fine-tuning?tabs=azure-openai%2Cturbo%2Cpython-new&pivots=programming-language-python

In [18]:
import os
import json
import pandas as pd

In [19]:
# Reading in the answers
# Dataset shortened to 1 lakh rows

df_answers = pd.read_csv('reddit_answers_big.csv', sep=';')
df_answers.head(3)

,Unnamed: 0,q_id,text,votes
0,0,hvbvpz,Two pet ducks. You may be tempted to go for on...,2359.0
1,1,hvbvpz,Nice try Jeff Bezos,764.0
2,2,hvbvpz,A curved shower rod. Seriously. $10 for a tens...,1525.0


In [20]:
# Grabbing the top answers
df_top_votes = df_answers.groupby('q_id')['votes'].idxmax()
df_top_answers = df_answers.loc[df_top_votes]

df_top_answers.rename(columns={'text': 'answer'}, inplace=True)
df_top_answers.rename(columns={'q_id': 'id'}, inplace=True)
df_top_answers.rename(columns={'votes': 'answer_votes'}, inplace=True)

df_top_answers.head(3)

,Unnamed: 0,id,answer,answer_votes
96052,99426,1004g5,Blow Me Away by Breaking Benjamin http://www....,7.0
98166,101623,105i80,Move out. Don't tell your family where you mov...,983.0
97972,101424,12gj2x,There's a Canadian town called Lloydminster th...,456.0


In [21]:
# Reading in the questions
# Dataset shortened to 1 lakh rows

df_questions = pd.read_csv('reddit_questions.csv', sep=';')

df_questions.rename(columns={'text': 'question'}, inplace=True)
df_questions.rename(columns={'votes': 'question_votes'}, inplace=True)

df_questions.head(3)

,id,question,question_votes,timestamp,datetime
0,izucgz,What's the purpose of life?,8,1.601076e+09,Fri Sep 25 23:13:31 2020 UTC
1,9c784/,"I've tried to quit smoking, this is my seventh...",11,1.250712e+09,Wed Aug 19 19:58:54 2009 UTC
2,iylxwl,"For those who have a slave master last name, w...",0,1.600904e+09,Wed Sep 23 23:35:15 2020 UTC


In [22]:
merged_df = df_questions.merge(df_top_answers, on='id')

merged_df.head(3)

,id,question,question_votes,timestamp,datetime,Unnamed: 0,answer,answer_votes
0,crwzh/,Upload a piece of your life and make other peo...,9,1.279689e+09,Wed Jul 21 05:12:32 2010 UTC,99287,Here's mine,4.0
1,ne2rg/,Black Redditors - Whats your most awkward raci...,1107,1.323975e+09,Thu Dec 15 18:44:51 2011 UTC,102022,"""So I let a friend, who happened to be black (...",1839.0
2,coedjg,Zeus has decided that Olympus feels a little e...,51300,1.565417e+09,Sat Aug 10 06:05:51 2019 UTC,80496,God of Silence. You can pray to me to shut the...,35780.0


In [23]:
# Clean this data up a bit
merged_df.drop(['timestamp', 'datetime', 'Unnamed: 0'], axis=1, inplace=True)

merged_df = merged_df.reindex(columns=['id', 'question',  'answer','question_votes', 'answer_votes'])

merged_df.head(3)

,id,question,answer,question_votes,answer_votes
0,crwzh/,Upload a piece of your life and make other peo...,Here's mine,9,4.0
1,ne2rg/,Black Redditors - Whats your most awkward raci...,"""So I let a friend, who happened to be black (...",1107,1839.0
2,coedjg,Zeus has decided that Olympus feels a little e...,God of Silence. You can pray to me to shut the...,51300,35780.0


In [24]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590 entries, 0 to 589
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              590 non-null    object 
 1   question        590 non-null    object 
 2   answer          590 non-null    object 
 3   question_votes  590 non-null    int64  
 4   answer_votes    590 non-null    float64
dtypes: float64(1), int64(1), object(3)
memory usage: 23.2+ KB


In [25]:
# Fine-tining format
questions, answers = merged_df['question'], merged_df['answer']

In [26]:
qa_openai_format = [{"messages":[{"role":"system", "content": "Marv is a factual chatbot and reddit expert who likes to answer with bullets."},
                                 {"role": "user", "content": q},
                                 {"role": "assistant", "content": a}]} for q, a in zip(questions, answers)]

qa_openai_format[5]

{'messages': [{'role': 'system',
   'content': 'Marv is a factual chatbot and reddit expert who likes to answer with bullets.'},
  {'role': 'user',
   'content': "0.1% doesn't seem much, however, What would horribly, catastrophically, go wrong if it was off by 0.1%?"},
  {'role': 'assistant',
   'content': 'If 0.1% of all airline flights crashed, there would be around 87 crashes every day.'}]}

In [27]:
with open("training_data.jsonl", "w") as f:
    for entry in qa_openai_format:
        f.write(json.dumps(entry))
        f.write("\n")

In [28]:
from collections import defaultdict

data_path = "training_data.jsonl"

# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
  dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))

Num examples: 590


In [32]:
# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue
        
    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue
        
    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1
        
        if any(k not in ("role", "content", "name", "function_call") for k in message):
            format_errors["message_unrecognized_key"] += 1
        
        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1
            
        content = message.get("content", None)
        function_call = message.get("function_call", None)
        
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1
    
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1   

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


In [39]:
import os
from openai import AzureOpenAI

In [73]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [74]:
os.getenv('AZURE_OPENAI_ENDPOINT')

'https://vasub-m63b076p-southcentralus.cognitiveservices.azure.com'

In [75]:
client = AzureOpenAI(
    api_key=os.getenv('OPENAI_API_KEYYY'),
    api_version=os.getenv('OPENAI_API_VERSION'),
    azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT')
)

In [76]:
# Sending data to OpenAI
client.files.create(
    file = open('training_data.jsonl', 'rb'),
    purpose = 'fine-tune'
)

FileObject(id='file-12112b1f7e984b86a696311c2001db24', bytes=302281, created_at=1737272356, filename='training_data.jsonl', object='file', purpose='fine-tune', status='pending', status_details=None)

In [78]:
# Creating a simple deployment is enough for this step. But check documentation before creating any model. 
# Only few models with very specific versions can do fine-tuning. 
# https://learn.microsoft.com/en-us/azure/ai-studio/concepts/fine-tuning-overview
# No quota available for the currently supported regions!

client.fine_tuning.jobs.create(
    training_file='file-12112b1f7e984b86a696311c2001db24',
    model= "gpt-35-turbo"
)

BadRequestError: Error code: 400 - {'error': {'code': 'invalidPayload', 'message': 'The specified base model does not support fine-tuning.'}}

In [ ]:
# This gives us the name of the fine tuned model
client.fine_tuning.jobs.list()

In [79]:
system_prompt = "Marv is a factual chatbot and reddit expert who likes to answer with bullets"
user_question = "Give me the dumbest thing you've ever done."

In [80]:
response = client.chat.completions.create(
    model = "ft:gpt-3.5-turbo-0613:personal::8WsMPAem",
    messages = [
        {'role':'system', "content": system_prompt},
        {'role':'user', "content": user_question}
    ]
)

print(response.choices[0].message.content)

AuthenticationError: Error code: 401 - {'statusCode': 401, 'message': 'Unauthorized. Access token is missing, invalid, audience is incorrect (https://cognitiveservices.azure.com), or have expired.'}

In [81]:
# Interestingly enough fine-tuning works with the Azure portal - no regions constraints?
# Started a test case or will it be stuck on queue? Let's see - Maybe the UX checks are missing